# Convolutional Neural Networks (CNN)
* 컨볼루션 신경망(CNN)은 컨볼루션 레이어와 풀링을 결합한 신경망이다.  
* Gradient-based learning applied to document recognition, Y. Lecun et al., (1998). [https://ieeexplore.ieee.org/document/726791]

### MNIST example

In [25]:
import numpy as np
from tensorflow import keras
from keras import models 
from keras import layers
from keras.datasets import mnist
from keras.utils import to_categorical
import matplotlib.pyplot as plt

In [11]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# preprocessing
X_train = X_train.reshape((60000, 28, 28, 1))
X_train = X_train/255

X_test = X_test.reshape((10000, 28, 28, 1))
X_test = X_test/255

y_train=to_categorical(y_train)
y_test=to_categorical(y_test)

In [12]:
net = models.Sequential()
net.add(layers.Conv2D(32,(3,3), activation='relu', input_shape=(28,28,1)))
net.add(layers.MaxPooling2D((2,2)))
net.add(layers.Conv2D(64,(3,3), activation='relu')) 
net.add(layers.MaxPooling2D((2,2)))
net.add(layers.Conv2D(64,(3,3), activation='relu'))
net.add(layers.Flatten())
net.add(layers.Dense(64, activation='relu')) 
net.add(layers.Dense(10, activation='softmax')) 
net.compile(optimizer='nadam',loss='categorical_crossentropy',metrics=['acc'])

In [13]:
net.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)               

In [14]:
net.fit(X_train, y_train, epochs=5, batch_size=64)

Epoch 1/5
938/938 [==============================] - 15s 16ms/step - loss: 0.1671 - acc: 0.9475
Epoch 2/5
938/938 [==============================] - 16s 17ms/step - loss: 0.0453 - acc: 0.9862
Epoch 3/5
938/938 [==============================] - 16s 18ms/step - loss: 0.0327 - acc: 0.9893
Epoch 4/5
938/938 [==============================] - 16s 18ms/step - loss: 0.0252 - acc: 0.9922
Epoch 5/5
938/938 [==============================] - 16s 17ms/step - loss: 0.0204 - acc: 0.9932


In [15]:
test_loss, test_acc = net.evaluate(X_test,y_test)

313/313 [==============================] - 1s 4ms/step - loss: 0.0229 - acc: 0.9929


### CIFAR-10 example
* https://www.cs.toronto.edu/~kriz/cifar.html

In [17]:
from keras.datasets import cifar10

(X_train, y_train), (X_test, y_test) = cifar10.load_data()
# it takes a few minutes..

In [23]:
# preprocessing
X_train = X_train.reshape((50000, 32, 32, 3))
X_train = X_train/255

X_test = X_test.reshape((10000, 32, 32, 3))
X_test = X_test/255

y_train=to_categorical(y_train)
y_test=to_categorical(y_test)

In [27]:
import matplotlib.pyplot as plt

plt.subplot(141)
plt.imshow(X_train[0])
plt.subplot(142)
plt.imshow(X_train[1])
plt.subplot(143)
plt.imshow(X_train[2]
plt.subplot(144)
plt.imshow(X_train[3]

SyntaxError: invalid syntax (<ipython-input-27-7d1bad46de58>, line 9)